<a href="https://colab.research.google.com/github/rudedev/examplo-crew-ai/blob/main/exemplo_crew_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U crewai crewai_tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [8]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

Agentes: 1 - Buscador de Conteúdo
2 - Redator de Conteúdo
3 - Editor de Conteúdo

In [11]:
buscador = Agent(
    role='Buscador de Contéudo',
    goal='Buscar conteúdo online sobre {tema}',
    backstory='Você está trabalhando na criação de artigos para o Linkedin sobre {tema}.'
              'Você vai fazer uma busca sobre informações na internet, coletá-las e agrupá-las.'
              'Seu trabalho servirá de base para o Redator de Conteúdo.',
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [12]:
redator = Agent(
    role='Redator de Conteúdo',
    goal='Escrever um texto divertido e factualmente correto para o Linkedin sobre {tema}',
    backstory='Você está trabalhando na redação de um artigo para o Linkedin sobre {tema}.'
              'Você vai utilizar os dados coletados pelo Buscador de Conteúdo para escrever um texto'
              'interessante, divertido e factualmente correto para o Linkedin.'
              'Dê opiniões sobre o {tema}, mas ao fazê-lo, deixe claro que são opiniões pessoais.',
    tools=[search_tool, scrape_tool],
    verbose=True
)

In [13]:
editor = Agent(
    role='Editor de Conteúdo',
    goal='Editar um texto de Linkedin para que ele tenha um tom mais informal.',
    backstory='Você está trabalhando na edição de um artigo para o Linkedin.'
              'Você vai receber um texto do Redator de Conteúdo, e editá-lo para o tom de voz do '
              'do Rude Developer que é mais informal e divertido.',
    tools=[search_tool, scrape_tool],
    verbose=True
)

Tarefas: 1 - Buscar, 2 - Redigir/Escrever, 3 - Editar

In [14]:
buscar = Task(
    description=(
      "1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre {tema}.\n"
      "2. Identifique o público-alvo, considerando seus interesses e pontos de dor.\n"
      "3. Inclua palavras-chave de SEO e dados ou fontes relevantes."),
    agent=buscador,
    expected_output='Um plano de tendências sobre {tema}, com as palavras mais relevantes de SEO e as últimas notícias.'
)

In [15]:
redigir = Task(
    description=(
        "1. Use os dados coletados de conteúdo para criar um post de LinkedIn atraente sobre {tema}.\n"
        "2. Incorpore palavras-chave de SEO de forma natural.\n"
        "3. Certifique-se de que o post esteja estruturado de forma cativante, com uma conclusão que faça o leitor refletir.\n"
     ),
    agent=redator,
    expected_output='Um texto de LinkedIn sobre {tema}.'
)

In [16]:
editar = Task(
    description=("Revisar a postagem de LinkedIn em questão quanto a erros gramaticais e alinhamento com a voz pessoal do Rude Developer."),
    agent=editor,
    expected_output='Um texto de LinkedIn pronto para publicação, seguindo o tom de voz esperado.'
                    'O texto está separado em parágrafos e não usa bullet points.'
)

Crew - Equipe

In [17]:
equipe = Crew(
    agents=[buscador, redator, editor],
    tasks=[buscar, redigir, editar],
    verbose=True
)

Rodando o Crew

In [18]:
tema_do_artigo = 'Alimentação Saudável'

entradas = {"tema":tema_do_artigo}

resultado = equipe.kickoff(inputs=entradas)

# Agent: Buscador de Contéudo
## Task: 1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre Alimentação Saudável.
2. Identifique o público-alvo, considerando seus interesses e pontos de dor.
3. Inclua palavras-chave de SEO e dados ou fontes relevantes.


# Agent: Buscador de Contéudo
## Thought: Vamos começar pela pesquisa das últimas tendências e notícias relevantes sobre Alimentação Saudável, juntamente com a identificação do público-alvo e palavras-chave de SEO.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"tend\\u00eancias Alimenta\\u00e7\\u00e3o Saud\\u00e1vel 2023 not\\u00edcias relevantes\"}"
## Tool Output: 

Search results: Title: Relatório NutriConnection traz as tendências de alimentação
Link: https://www.foodconnection.com.br/especialistas/relatorio-nutriconnection-traz-tendencias-de-alimentacao-saudavel-para-2023
Snippet: Suplementos: um grande destaque foi o aumento no consumo de suplementos e pós, devido a

In [19]:
from IPython.display import Markdown
Markdown(resultado.raw)

🌱 **Tendências em Alimentação Saudável para 2023**: Um Mergulho na Vida Equilibrada! 🌱

Já parou pra pensar em como a **alimentação saudável** tá mudando e se adaptando às loucuras do mundo moderno? Se a resposta foi um não bem sonoro, bora lá dar uma espiadinha! Este ano tá cheio de tendências sensacionais que não apenas buscam o bem-estar físico e mental, mas também dão aquele help na sustentabilidade do nosso planeta. Vamos nessa?

🥗 **1. Dieta Flexitariana**: Olha, a flexibilidade é a palavra da vez! A dieta flexitariana, que combina uma base vegetarianezinha com um tiquinho de carne de vez em quando, tá bombando! É prática, cheia de sabor e até os carnívoros vão curtir! E, sério, diminuir o consumo de carne faz uma diferença enorme no meio ambiente, viu?

🤝 **2. Bem-Estar Equilibrado**: Saúde não é só ficar bombando na academia; é mental e emocional também, mermão! Esse conceito ganhou força e traz a ideia de um estilo de vida que contagiando, em vez de restringir. Que tal arriscar meditação, alongamentos e, claro, refeições que realmente façam seus olhos brilharem?

🌍 **3. Transparência na Produção**: Nos tempos de hoje, saber de onde vem a comida que a gente coloca no prato é mais importante do que nunca! A transparência dos produtores tá em alta e quem não valoriza isso tá perdendo um ponto crucial. Produção ética é o novo pretinho básico!

🐾 **4. Nutrição para Pets**: E aí, quem disse que nossos amigos peludos não merecem uma comida saudável também? A galera tá cada vez mais ligada na nutrição dos pets, e a saúde deles influencia a nossa felicidade, não é mesmo?

🌶️ **5. Experimentação de Novos Sabores**: Se você sempre teve medo de se jogar na cozinha e preparar algo diferente, esse é o seu momento, amigo! Usar especiarias, ingredientes diferentes e lançar combinações inusitadas tá na moda. Seja ousado e se permita descobrir novos paladares!

**Conclusão**: Refletindo sobre essas novas tendências, fica claro que a **alimentação saudável** vai muito além do que botamos na boca; é sobre como fazemos nossas escolhas e como elas afetam o nosso mundão. E você, tá pronto pra dar essa guinada na sua vida? Vamo que vamo em busca de uma vida mais equilibrada e saborosa, galera!

🔗 **#AlimentacaoSaudavel #DietaFlexitariana #Sustentabilidade #NovosSabores** 

Deixa seu comentário e compartilha suas experiências com a alimentação saudável! 🍽️✨